In [1]:
import os
import tkinter as tk
from tkinter import messagebox, scrolledtext
from PIL import Image, ImageTk
import time
import random
from bs4 import BeautifulSoup
import json
import requests

class SpatialQuizApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Spatial Reasoning Quiz")
        self.root.geometry("800x600")
        self.user_id = None
        self.pretest_gender = None
        self.pretest_age = None
        self.score = 0

        self.create_start_page()

    def create_start_page(self):
        # Welcome message and instructions
        welcome_label = tk.Label(self.root, text="Welcome to the Spatial Reasoning Quiz!", font=("Helvetica", 16, "bold"))
        welcome_label.pack(pady=20)

        instructions_text = """
        Instructions:
        This quiz tests your spatial reasoning abilities.
        You will be shown images and asked questions about their spatial arrangements.
        Please read the data consent information and enter 'yes' to proceed.
        After consent, you'll be asked to generate a 4-letter unique user identifier.
        This identifier will be used to link your responses anonymously.
        Enjoy the quiz!
        """
        instructions_label = scrolledtext.ScrolledText(self.root, wrap=tk.WORD, width=80, height=10)
        instructions_label.insert(tk.END, instructions_text)
        instructions_label.config(state=tk.DISABLED)
        instructions_label.pack(pady=10)

        # Button to start the quiz
        start_button = tk.Button(self.root, text="Start Quiz", command=self.show_data_consent)
        start_button.pack(pady=10)

    def show_data_consent(self):
        for widget in self.root.winfo_children():
            widget.destroy()
        self.create_data_consent()
#Displaying the data consent page
    def create_data_consent(self):
        data_consent_info = """
            DATA CONSENT INFORMATION:
            Please read:
            We wish to record your response data
            to an anonymised public data repository.
            Your data will be used for educational teaching purposes
            practising data analysis and visualisation.
            Please type yes in the box below if you consent to the upload.
            Please contact philip.lewis@ucl.ac.uk
            If you have any questions or concerns
            regarding the stored results
            """
        data_consent_label = scrolledtext.ScrolledText(self.root, wrap=tk.WORD, width=80, height=10)
        data_consent_label.insert(tk.END, data_consent_info)
        data_consent_label.config(state=tk.DISABLED)
        data_consent_label.pack(pady=10)
        # Entry for user consent
        consent_entry = tk.Entry(self.root)
        consent_entry.pack(pady=10)

        # Button
        consent_button = tk.Button(self.root, text="Continue", command=lambda: self.process_consent(consent_entry.get()))
        consent_button.pack(pady=10)
    def process_consent(self, consent_response):
        if consent_response.lower() == "yes":
            # Hide the consent section and proceed to load the quiz
            self.hide_consent_widgets()
        else:
            # Close the application or handle non-consent
            self.root.destroy()

    def hide_consent_widgets(self):
        # Hide data consent section
        for widget in self.root.winfo_children():
            widget.destroy()
        self.generate_user_id ()
        
    def generate_user_id(self):
        id_instructions = """
        Enter your anonymized ID
        To generate an anonymous 4-letter unique user identifier, please enter:
            - two letters based on the initials (first and last name) of a childhood friend
            - two letters based on the initials (first and last name) of a favorite actor/actress
            
        For example, if your friend was called Charlie Brown and the film star was Tom Cruise,
        then your unique identifier would be CBTC.
        """
        
        id_label = tk.Label(self.root, text=id_instructions, wraplength=400, justify=tk.LEFT)
        id_label.pack(pady=10)

        user_id_entry = tk.Entry(self.root)
        user_id_entry.pack(pady=10)

        submit_button = tk.Button(self.root, text="Submit", command=lambda: self.process_user_id(user_id_entry.get()))
        submit_button.pack(pady=10)

    def process_user_id(self, user_id):
        if len(user_id) == 4:
            # Storing user id
            self.user_id = user_id
        else:
            # Handle invalid user ID (e.g., show a message)
            messagebox.showwarning("Invalid User ID", "Please enter a 4-letter user ID.")
        for widget in self.root.winfo_children():
            widget.destroy()
        self.generate_pretest_gender ()

    def generate_pretest_gender(self):
        gender_instructions = """
        Enter your gender (Female/Male/Not Prefer to Say)
        """

        gender_label = tk.Label(self.root, text=gender_instructions, wraplength=400, justify=tk.LEFT)
        gender_label.pack(pady=10)

        gender_entry = tk.Entry(self.root)
        gender_entry.pack(pady=10)

        submit_button = tk.Button(self.root, text="Submit", command=lambda: self.process_pretest_gender(gender_entry.get()))
        submit_button.pack(pady=10)

    def process_pretest_gender(self, pretest_gender):
        # Storing pretest data
        self.pretest_gender = pretest_gender

        # Remove old widget
        for widget in self.root.winfo_children():
            widget.destroy()
        self.generate_pretest_age()

    def generate_pretest_age(self):
        age_instructions = """
        Enter your age
        """

        age_label = tk.Label(self.root, text=age_instructions, wraplength=400, justify=tk.LEFT)
        age_label.pack(pady=10)

        age_entry = tk.Entry(self.root)
        age_entry.pack(pady=10)

        submit_button = tk.Button(self.root, text="Submit", command=lambda: self.process_pretest_age(age_entry.get()))
        submit_button.pack(pady=10)

    def process_pretest_age(self, pretest_age):
        # Storing pretest data
        self.pretest_age = pretest_age

        # Remove old widgets
        for widget in self.root.winfo_children():
            widget.destroy()
        self.load_quiz()
        
    def load_quiz(self):
        #For the actual quiz
        self.question_label = tk.Label(self.root, text="Question 1")
        self.question_label.pack(pady=10)

        self.image_label = tk.Label(self.root)
        self.image_label.pack(pady=10)

        options_frame = tk.Frame(self.root)
        options_frame.pack()
 
        #questions list
        self.questions = [
            "Question 1: What would be the view, if the 3D arrangement of cubes was seen from the top?",
            "Question 2: What would the 3D arrangement of cubes look like if it was flipped upside down?",
            "Question 3: What would the 3D arrangement of cubes look like if it was seen from the left?",
            "Question 4: What would the 3D arrangement of cubes look like if seen from the right?",
            "Question 5: If rotated 180 degrees, what would the 3D arrangement of cubes look like if seen from the bottom?",
            "Question 6: If rotated 180 degrees, what would the 3D arrangement of cubes look like from the right?",
            "Question 7: What would the 3D arrangement of cubes look like if seen from the top?",
            "Question 8: What would be the arrangement of cubes look like if seen from the right?",
            "Question 9: What would be the arrangement of cubes look like if seen from the top?",
            "Question 10: If rotated 180 degrees, what would the 3D arrangement of cubes look like from the left?"
        ]
        #the answers
        self.correct_answers = [0, 2, 0, 0, 3, 2, 0, 2, 3, 1]

        self.answers = []
        for i in range(1, 11):
            question_image = self.load_image(os.path.join("FINAL_SpatialTask", f"Q{i}ref.png"))
            self.answers.append(question_image)

            for option in ["a", "b", "c", "d"]:
                answer_image = self.load_image(os.path.join("FINAL_SpatialTask", f"Q{i}{option}.png"))
                self.answers.append(answer_image)

        self.current_question = 0
        self.score = 0
        self.start_time = 0

        self.create_widgets()

    def create_widgets(self):
        self.question_label = tk.Label(self.root, text="Question 1")
        self.question_label.pack(pady=10)

        self.image_label = tk.Label(self.root)
        self.image_label.pack(pady=10)

        options_frame = tk.Frame(self.root)
        options_frame.pack()

        self.option_buttons = []
        for i, option in enumerate(["a", "b", "c", "d"]):
            btn = tk.Button(options_frame, text=option.upper(), command=lambda o=option: self.check_answer(o))
            btn.grid(row=i // 2, column=i % 2, pady=5, padx=5)
            self.option_buttons.append(btn)

        self.load_question()

    def load_question(self):
        self.question_label.config(text=self.questions[self.current_question])
        self.start_time = time.time()

        # The reference question image - 3D cubes arrangement
        question_image = ImageTk.PhotoImage(self.answers[self.current_question * 5])
        self.image_label.config(image=question_image)
        self.image_label.image = question_image

        # Each answer option button - WITH THE IMAGES ON THEM
        for i, option in enumerate(["a", "b", "c", "d"]):
            answer_image = ImageTk.PhotoImage(self.answers[self.current_question * 5 + i + 1])
            self.option_buttons[i].config(image=answer_image, compound=tk.LEFT)
            self.option_buttons[i].image = answer_image

    def check_answer(self, selected_option):
        elapsed_time = time.time() - self.start_time

        # Checking selected answer is correct
        correct_option = chr(ord('a') + self.correct_answers[self.current_question])
        if selected_option == correct_option:
            self.score += 1
        else:
            self.score -= 0.25

        if self.current_question < 9:
            self.current_question += 1
            self.load_question()
        else:
            self.show_result()

    def load_image(self, path):
        if os.path.exists(path):
            img = Image.open(path)
            img = img.resize((300, 200), Image.LANCZOS)
            return img
        else:
            raise FileNotFoundError(f"File not found: {path}")

    def show_result(self):
            # Upload data to Google Form
        data_dict = {
            'User ID': self.user_id,
            'Gender': self.pretest_gender,
            'Age': self.pretest_age,
            'Score': self.score
            }

        form_url = "https://docs.google.com/forms/d/e/1FAIpQLSeRf4uJIOdZf7CapR7UgwVlSa2Mbc8tiMigCpJD_763Smvvvg/viewform?usp=sf_link"
        upload_success = self.send_to_google_form(data_dict, form_url)

            # Print the result
        if upload_success:
            print("Data uploaded successfully!")
        else:
            print("Data upload failed.")

        messagebox.showinfo("Quiz Completed", f"Your score: {self.score}")
        self.root.destroy()

    def send_to_google_form(self, data_dict, form_url):
        ''' Helper function to upload information to a corresponding Google Form
            You are not expected to follow the code within this function!
        '''
        form_id = form_url[34:90]
        view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
        post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'
        page = requests.get(view_form_url)
        content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
        if content is None:
            print("Script tag not found. Data not uploaded.")
            return False

        content_text = content.text[27:-1] if content.text else None

        if not content_text:
            print("Content text not found. Data not uploaded.")
            return False

        try:
            result = json.loads(content_text)[1][1]
            print("Content of the script tag:")
            print(result)
            form_dict = {}
        
            loaded_all = True
            for item in result:
                # Remove non-alphanumeric characters from the item name
                item_name_cleaned = ''.join(char.lower() for char in item[1] if char.isalnum())
                
                # Check if the cleaned item name matches any key in the data dictionary
                matching_keys = [key for key in data_dict.keys() if item_name_cleaned in ''.join(char.lower() for char in key if char.isalnum())]
                
                if not matching_keys:
                    print(f"Form item {item[1]} not found. Data not uploaded.")
                    loaded_all = False
                    return False
                # Use the first matching key as the data dictionary key
                form_dict[f'entry.{item[4][0][0]}'] = data_dict[matching_keys[0]]
            
            post_result = requests.post(post_form_url, data=form_dict)
            return post_result.ok
        
        except (json.JSONDecodeError, IndexError) as e:
            print(f"Error decoding JSON or accessing result: {e}")
            return False

if __name__ == "__main__":
    root = tk.Tk()
    app = SpatialQuizApp(root)
    root.mainloop()
